In [112]:
# Importing the required Libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [113]:
# defining the topics for the news
topics = ['sports','science','technology','india','world','corona','economy']

In [12]:
# The instance of Firefox WebDriver is created.
driver = webdriver.Firefox(executable_path = 'C:\Webdr\geckodriver.exe')
wait = WebDriverWait(driver, 10)

In [115]:
# Link of the site from which news are being scraped
link = "https://www.reuters.com"
links = set() # set for collecting links of the articles

for topic in topics: 
    # The driver.get method will navigate to a page given by the URL
    driver.get(link + '/search/news?blob=' + topic)
    
    page_num = 0
    while page_num < 100:
        try:
            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'search-result-more-txt')))
            driver.find_element_by_class_name('search-result-more-txt').click()
            page_num += 1
        except:
            break
            
    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html,'html.parser')
    for i in soup.find_all('h3', attrs = {'class':'search-result-title'}):
        links.add(link + i.a.get('href'))

In [140]:
articles_info = []
for link in links:
    single_response = requests.get(link)
    single_html_data = single_response.content
    single_soup = BeautifulSoup(single_html_data, 'html.parser')
    articles_dict = {}
    articles_dict['link'] = link
    articles_dict['title'] = single_soup.find('h1', attrs = {'class': 'Headline-headline-2FXIq Headline-black-OogpV ArticleHeader-headline-NlAqj'}).get_text()
    text = ""
    for each_tag in single_soup.find_all('p', attrs = {'class': "Paragraph-paragraph-2Bgue ArticleBody-para-TD_9x"}):
        text += each_tag.get_text()
    articles_dict['text'] = text
    articles_info.append(articles_dict)

In [141]:
# making the dataframe of extracted news
news = pd.DataFrame(articles_info)

In [142]:
# writing the news DataFrame into Excel File
writer = pd.ExcelWriter('scrapednews.xlsx', engine='xlsxwriter')
news.to_excel(writer, sheet_name='Sheet1')
writer.save()

In [ ]:
# or you can write it into a csv file
news.to_csv('scraped_news.csv')